# 有価証券報告書「事業等のリスク」セクション抽出処理

  概要
	•	対象データ:
edinet_documents テーブルに登録された**有価証券報告書（書類種別コード: 120）**のドキュメントID一覧を対象とします。
	•	目的:
各企業の「事業等のリスク」セクションを抽出・構造化し、研究・分析に活用可能な JSON 形式で保存します。

⸻

 処理フロー

1. DB接続し対象書類を取得
	•	PostgreSQL に接続し、対象となる有価証券報告書（書類種別コード 120）を抽出します。
	•	抽出情報: doc_id, company_id, edinet_code, fiscal_year, submit_date など。

2. CSVから抽出
	•	ローカルに該当書類の CSV（EDINET開示文書CSV形式）が存在する場合、これを優先使用。
	•	「事業等のリスク」セクションを 見出し単位で抽出。

3. XBRLから抽出（フォールバック処理）
	•	CSVが存在しない場合、EDINET API v2から XBRL を取得。
	•	ZIP 解凍後、XBRL 文書から 「事業等のリスク」テキストブロックを抽出。

4. リスクセクションの検出・分離
	•	テキストブロックから「事業等のリスク」セクションを抽出。
	•	見出しパターンを正規化し、次の見出しまでを本文とする。

5. JSON形式で保存
	•	data/processed/risk_sections/{fiscal_year}/{company_id}_{doc_id}.json の形式で保存。
	•	JSONには以下の情報を含めます：
	•	company_id
	•	doc_id
	•	fiscal_year
	•	submit_date
	•	risk_text

⸻

  実装方針
	•	ログ管理: logging モジュールにより INFO レベルで詳細ログを取得（デフォルト表示は抑制）。
	•	進捗表示: tqdm により、抽出処理の進行状況を表示。
	•	再現性: 各ステップを関数化・モジュール化し、Notebook内にセルで分割配置。

## 1. 環境設定とデータベース接続

In [1]:
import sys
import os

os.chdir(os.path.abspath(".."))
sys.path.append(os.path.abspath(".."))

In [6]:
# ライブラリのインポート
import os
import re
import json
import zipfile
import io
import time
import logging
import requests
import pandas as pd
from tqdm.notebook import tqdm
from sqlalchemy import create_engine
from dotenv import load_dotenv

load_dotenv()

# ログ設定: INFOレベルのログを抑制しWARNING以上のみ表示
logging.basicConfig(level=logging.WARNING, format='%(asctime)s [%(levelname)s] %(message)s')
logger = logging.getLogger(__name__)

# データベース接続情報（環境変数や設定ファイルから読み込むことを推奨）
DB_USER = os.getenv('POSTGRES_USER')
DB_PASS = os.getenv('POSTGRES_PASSWORD')
DB_HOST = os.getenv('POSTGRES_HOST')
DB_PORT = os.getenv('POSTGRES_PORT')
DB_NAME = os.getenv('POSTGRES_DB')

# SQLAlchemyエンジンの作成
db_url = f"postgresql+psycopg2://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(db_url)

# EDINET APIキーの設定（環境変数から取得）
EDINET_API_KEY = os.getenv('EDINET_API_KEY')
if not EDINET_API_KEY:
    logger.warning("EDINET APIキーが設定されていません。環境変数EDINET_API_KEYを設定してください。")

## 2. 抽出対象書類の取得

In [7]:
# 有価証券報告書(doc_type_code=120)を対象に書類情報を取得
query = """
    SELECT doc_id, company_id, edinet_code, doc_type_code, submit_date, fiscal_year, description, local_csv_path
    FROM edinet_documents
    WHERE doc_type_code = '120'
    ORDER BY submit_date;
"""
documents_df = pd.read_sql(query, engine)
print(f"取得した書類数: {len(documents_df)} 件")
documents_df.head(5)

取得した書類数: 8663 件


,doc_id,company_id,edinet_code,doc_type_code,submit_date,fiscal_year,description,local_csv_path
0,S1009HM0,128c49ff-7f18-49ca-9fc2-5a5cb0ec3a24,E25682,120,2017-01-25 15:15:00,2017,有価証券報告書－第13期(2015/11/01－2016/10/31),data/raw/edinet_csv_data/2017/E25682/S1009HM0.csv
1,S1009I5F,c9cc96eb-b1fe-47e5-b82a-f79671981ae4,E31424,120,2017-01-27 13:12:00,2017,有価証券報告書－第11期(2015/11/01－2016/10/31),data/raw/edinet_csv_data/2017/E31424/S1009I5F.csv
2,S1009HL6,6321451e-8c3e-42c7-9131-f0aaf4e52be6,E04979,120,2017-01-27 15:52:00,2017,有価証券報告書－第32期(2015/11/01－2016/10/31),data/raw/edinet_csv_data/2017/E04979/S1009HL6.csv
3,S1009IHZ,0c1107b6-cc06-43aa-86de-b4bae542db75,E30881,120,2017-01-30 15:01:00,2017,有価証券報告書－第15期(2015/11/01－2016/10/31),data/raw/edinet_csv_data/2017/E30881/S1009IHZ.csv
4,S1009IJ2,54a8a765-c69e-4bf5-b6fd-5ff567e8a523,E02999,120,2017-01-30 16:46:00,2017,有価証券報告書－第31期(2015/11/01－2016/10/31),data/raw/edinet_csv_data/2017/E02999/S1009IJ2.csv


## 3. CSVファイルから「事業等のリスク」テキスト抽出

In [8]:
def extract_risk_from_csv(csv_path: str) -> str:
    """有価証券報告書のCSVから「事業等のリスク」テキストを抽出する"""
    if not os.path.exists(csv_path):
        return None
    try:
        # CSV読み込み（UTF-16, TAB区切り）
        df = pd.read_csv(csv_path, encoding='utf-16', sep='\t')
    except Exception as e:
        logger.error(f"CSV読み込みエラー: {csv_path} - {e}")
        return None

    # BusinessRisksTextBlockに該当する行を抽出
    risk_rows = df[df["要素ID"] == "jpcrp_cor:BusinessRisksTextBlock"]
    if risk_rows.empty:
        # タクソノミ項目が見つからない場合
        logger.warning(f"CSV中にBusinessRisksTextBlockが見つかりません: {csv_path}")
        return None

    # 想定通り1行（または複数行の場合は先頭）の値を取得
    risk_text = str(risk_rows.iloc[0]["値"])
    if pd.isna(risk_text):
        logger.warning(f"CSV中のリスク項目値がNaNです: {csv_path}")
        return None

    # テキスト整形：前後の空白除去（必要に応じてタグ除去など追加処理可能）
    risk_text = risk_text.strip()
    return risk_text

# 関数のテスト（最初のドキュメントでCSVパスがあれば試す）
test_doc = documents_df.iloc[0]
if pd.notna(test_doc['local_csv_path']) and os.path.exists(test_doc['local_csv_path']):
    sample_text = extract_risk_from_csv(test_doc['local_csv_path'])
    print(f"サンプル抽出: {test_doc['doc_id']} のリスクテキスト冒頭:\n", sample_text[:100], "...")

サンプル抽出: S1009HM0 のリスクテキスト冒頭:
 ４【事業等のリスク】当社グループの事業、財務状態等、また投資者の判断に重要な影響を及ぼす可能性のあるリスクには以下のようなものがあります。なお、記載した事項は、当連結会計年度末現在において当社グループ ...


### 4. XBRLファイルから「事業等のリスク」テキスト抽出

In [10]:
def download_and_extract_xbrl(doc_id: str) -> bytes:
    """EDINET APIからdoc_idに対応するXBRLファイルを取得し、ZIP中のXBRLファイルを返す"""
    url = f"https://api.edinet-fsa.go.jp/api/v2/documents/{doc_id}"
    params = {"type": 1, "Subscription-Key": EDINET_API_KEY}
    try:
        res = requests.get(url, params=params)
        res.raise_for_status()
    except requests.RequestException as e:
        logger.error(f"EDINET APIリクエスト失敗: {doc_id} - {e}")
        return None
    # ZIPをバイナリデータとしてメモリ上で扱う
    try:
        with zipfile.ZipFile(io.BytesIO(res.content)) as z:
            # ZIP内のXBRLファイル（PublicDoc配下）を探す
            xbrl_files = [f for f in z.namelist() if f.endswith(".xbrl") and "PublicDoc" in f]
            if not xbrl_files:
                logger.warning(f"ZIP内にXBRLファイルが見つかりません: {doc_id}")
                return None
            xbrl_filename = xbrl_files[0]
            xbrl_content = z.read(xbrl_filename)  # XBRLファイル内容をバイト列で取得
            return xbrl_content
    except zipfile.BadZipFile as e:
        logger.error(f"ZIP解凍失敗: {doc_id} - {e}")
        return None

def extract_risk_from_xbrl(xbrl_content: bytes) -> str:
    """XBRLファイル内容から「事業等のリスク」テキストを抽出する"""
    try:
        text = xbrl_content.decode('utf-8')
    except UnicodeDecodeError:
        # XBRLは基本UTF-8のはずだが、失敗した場合はISO-8859-1等で再試行
        try:
            text = xbrl_content.decode('cp932')  # 一般的な日本語エンコーディング
        except Exception as e:
            logger.error(f"XBRLテキストのデコード失敗: {e}")
            return None

    # タグ名で抽出: BusinessRisksTextBlock要素を正規表現で抽出
    # (?s)で改行含めてマッチし、非貪欲で</...>まで取得
    match = re.search(r'(<jp[^:]*:BusinessRisksTextBlock[^>]*>.*?</jp[^:]*:BusinessRisksTextBlock>)', text, flags=re.DOTALL)
    if not match:
        logger.warning("BusinessRisksTextBlockタグがXBRL内に見つかりません")
        return None
    risk_block = match.group(1)

    # 抽出したテキストブロックからタグを除去してテキストのみ取得
    # まずタグを改行区切りに一旦置換（段落などを維持するため）
    risk_text = re.sub(r'</?(?:jp[^:]*:)?[^>]+>', '\n', risk_block)  # 開始・終了タグを改行に置換
    risk_text = risk_text.replace('&nbsp;', ' ')  # HTMLエンティティの簡易処理
    risk_text = risk_text.strip()
    return risk_text

# 関数のテスト: 最初のドキュメントでCSVが無い場合にXBRL抽出を試す
if pd.isna(test_doc['local_csv_path']) or not os.path.exists(test_doc['local_csv_path']):
    xbrl_data = download_and_extract_xbrl(test_doc['doc_id'])
    if xbrl_data:
        sample_text = extract_risk_from_xbrl(xbrl_data)
        print(f"サンプルXBRL抽出: {test_doc['doc_id']} のリスクテキスト冒頭:\n", sample_text[:100], "...")

### 5. テキストの正規化・見出し検出

In [16]:
import unicodedata

def normalize_risk_text(text: str) -> str:
    """リスクテキストをUnicode正規化し不要な文字を削除して整形する"""
    # Unicode正規化(NFKC): 全角数字や記号を半角に統一、合成文字の分解など
    norm_text = unicodedata.normalize("NFKC", text)
    norm_text = re.sub(r'[\r\t\f\v]', '', norm_text)
    # 重複する改行の削除（連続改行は1つに）
    norm_text = re.sub(r'\n\n+', '\n\n', norm_text)
    norm_text = norm_text.strip()
    #（必要に応じて）見出し部分「事業等のリスク」の除去
    if norm_text.startswith("事業等のリスク"):
        norm_text = norm_text[len("事業等のリスク"):].lstrip("：:。 \n")
    return norm_text

# テキスト整形のテスト
if 'sample_text' in locals():
    print("整形後テキスト冒頭:\n", normalize_risk_text(sample_text)[:100], "...")

整形後テキスト冒頭:
 4【事業等のリスク】当社グループの事業、財務状態等、また投資者の判断に重要な影響を及ぼす可能性のあるリスクには以下のようなものがあります。なお、記載した事項は、当連結会計年度末現在において当社グループ ...


### 6. 抽出処理の一括実行と結果保存

In [18]:
# 抽出結果を保存するフォルダを用意
output_base = "data/processed/risk_sections"
os.makedirs(output_base, exist_ok=True)

processed_count = 0
error_count = 0

for _, row in tqdm(documents_df.iterrows(), total=len(documents_df), desc="Processing"):
    doc_id = row['doc_id']
    company_id = row['company_id']
    edinet_code = row['edinet_code']
    fiscal_year = row['fiscal_year']
    submit_date = row['submit_date']
    doc_type = row['doc_type_code']
    description = row['description']
    csv_path = row['local_csv_path']

    # 出力ファイルパス組み立て
    out_dir = os.path.join(output_base, str(fiscal_year))
    os.makedirs(out_dir, exist_ok=True)
    out_path = os.path.join(out_dir, f"{company_id}_{doc_id}.json")

    # 既に出力ファイルが存在する場合はスキップ（上書き防止）
    if os.path.exists(out_path):
        logger.info(f"既に処理済みのためスキップ: {out_path}")
        continue

    try:
        # 1. CSVから抽出を試行
        risk_text = None
        if pd.notna(csv_path) and os.path.exists(csv_path):
            risk_text = extract_risk_from_csv(csv_path)
        # 2. CSVで取得できなかった場合、XBRLから抽出
        if not risk_text:
            xbrl_content = download_and_extract_xbrl(doc_id)
            if xbrl_content:
                # API呼び出し間隔の調整（例として2秒スリープ）
                time.sleep(2)  # ※必要に応じて調整 [oai_citation:21‡zenn.dev](https://zenn.dev/robes/articles/81e18fb389fe01#:~:text=if%20file.startswith%28,RequestException%20as%20e)
                risk_text = extract_risk_from_xbrl(xbrl_content)
        if not risk_text:
            logger.warning(f"リスクテキスト抽出失敗: doc_id={doc_id}")
            error_count += 1
            continue

        # 3. テキストの正規化
        risk_text_norm = normalize_risk_text(risk_text)

        # 4. JSON構造の作成
        output_data = {
            "company_id": company_id,
            "edinet_code": edinet_code,
            "doc_id": doc_id,
            "submit_date": submit_date.strftime("%Y-%m-%d") if submit_date is not None else None,
            "fiscal_year": fiscal_year,
            "doc_type_code": doc_type,
            "description": description,
            "risk_text": risk_text_norm
        }

        # 5. JSONファイルに保存 (ensure_ascii=Falseで日本語をそのまま出力)
        with open(out_path, 'w', encoding='utf-8') as f:
            json.dump(output_data, f, ensure_ascii=False, indent=2, default=str)
        processed_count += 1
        logger.info(f"保存完了: {out_path}")

    except Exception as e:
        logger.error(f"処理中に例外発生: doc_id={doc_id} - {e}")
        error_count += 1
        continue

print(f"処理完了: 正常処理={processed_count}件, エラー={error_count}件")

Processing:   0%|          | 0/8663 [00:00<?, ?it/s]

2025-07-02 11:13:16,900 [WARNING] ZIP内にXBRLファイルが見つかりません: S100CK3E
2025-07-02 11:13:16,901 [WARNING] リスクテキスト抽出失敗: doc_id=S100CK3E
2025-07-02 11:13:25,425 [WARNING] ZIP内にXBRLファイルが見つかりません: S100FE00
2025-07-02 11:13:25,425 [WARNING] リスクテキスト抽出失敗: doc_id=S100FE00
2025-07-02 11:13:25,974 [WARNING] CSV中にBusinessRisksTextBlockが見つかりません: data/raw/edinet_csv_data/2019/E03004/S100FOU2.csv
2025-07-02 11:13:29,323 [WARNING] BusinessRisksTextBlockタグがXBRL内に見つかりません
2025-07-02 11:13:29,324 [WARNING] リスクテキスト抽出失敗: doc_id=S100FOU2
2025-07-02 11:13:35,812 [WARNING] ZIP内にXBRLファイルが見つかりません: S100I759
2025-07-02 11:13:35,813 [WARNING] リスクテキスト抽出失敗: doc_id=S100I759
2025-07-02 11:13:39,594 [WARNING] CSV中にBusinessRisksTextBlockが見つかりません: data/raw/edinet_csv_data/2020/E05573/S100J3OB.csv
2025-07-02 11:13:42,890 [WARNING] BusinessRisksTextBlockタグがXBRL内に見つかりません
2025-07-02 11:13:42,891 [WARNING] リスクテキスト抽出失敗: doc_id=S100J3OB
2025-07-02 11:13:43,384 [WARNING] CSV中にBusinessRisksTextBlockが見つかりません: data/raw/edinet_csv_data/202

処理完了: 正常処理=7826件, エラー=61件
